# Evaluating a Chat Bot (Single-Turn)


Chat bots take conversation history into account when responding to a user. It's helpful to evalute this part of their behavior to make sure the user experience doesn't degrade a few turns in.

This notebook gives a brief outline of how to do so.

In [1]:
# %pip install -U langchain openai

In [2]:
import uuid

from langsmith import Client

uid = uuid.uuid4()
client = Client()

In [3]:
examples = [
    {
        "question": "For",
        "chat_history": [
            {
                "role": "user",
                "content": "Help me prepare for my debate",
            },
            {
                "role": "assistant",
                "content": "Sure thing, what is the topic of debate?",
            },
            {
                "role": "user",
                "content": "Use of technology in the classroom.",
            },
            {
                "role": "assistant",
                "content": "Great. Which side would you like to represent?",
            },
        ],
    },
    {
        "question": "Yes, that would be helpful.",
        "chat_history": [
            {
                "role": "user",
                "content": "I’m having trouble understanding cultural relativism.",
            },
            {
                "role": "assistant",
                "content": "It’s a crucial concept. Essentially, it’s about viewing other cultures without the lens of our own biases.",
            },
            {
                "role": "user",
                "content": "So, like not judging other cultures based on our own standards?",
            },
            {
                "role": "assistant",
                "content": "Exactly! It helps anthropologists to understand cultures on their own terms. Would you like a historical example?",
            },
        ],
    },
    {
        "question": "Was the American Revolution an influence too?",
        "chat_history": [
            {
                "role": "user",
                "content": "Can you explain the causes of the French Revolution?",
            },
            {
                "role": "assistant",
                "content": "Certainly. The causes are manifold including economic hardship, social inequality, and political incompetence.",
            },
            {
                "role": "user",
                "content": "The bourgeoisie were a part of the Third Estate, right?",
            },
            {
                "role": "assistant",
                "content": "Correct. They were part of the larger Third Estate which also included peasants and workers, and they were heavily taxed unlike the other estates.",
            },
            {"role": "user", "content": "How did Enlightenment ideas contribute?"},
            {
                "role": "assistant",
                "content": "Enlightenment ideas inspired the populace by promoting rights and equality, which directly challenged the established feudal system.",
            },
        ],
    },
    {
        "question": "And this applies to all actions, no matter the size?",
        "chat_history": [
            {
                "role": "user",
                "content": "I can’t wrap my head around Newton’s Third Law.",
            },
            {
                "role": "assistant",
                "content": "It states that for every action, there’s an equal and opposite reaction. Think of it as a force pair.",
            },
            {"role": "user", "content": "Like when a rocket launches?"},
            {
                "role": "assistant",
                "content": "Exactly! The rocket propels fuel downwards, and in turn, it is propelled upwards with equal force.",
            },
        ],
    },
    {
        "question": "When do I steer left?",
        "chat_history": [
            {"role": "user", "content": "Parallel parking is tricky."},
            {"role": "assistant", "content": "It can be. Would you like some tips?"},
            {"role": "user", "content": "Yes, please."},
            {
                "role": "assistant",
                "content": "Start by aligning your car beside the other car, leaving about 2-3 feet between them.",
            },
            {
                "role": "user",
                "content": "Then steer all the way to the right while backing up?",
            },
            {
                "role": "assistant",
                "content": "Yes, but when your front door passes the other car’s bumper, straighten the wheel and continue to reverse.",
            },
        ],
    },
]

In [4]:
dataset_name = f"Chat Bot Evals Single-Turn Example - {uid}"
dataset = client.create_dataset(dataset_name)
client.create_examples(
    inputs=examples,
    dataset_id=dataset.id,
)

## 2. Define Custom Evaluator

Let's say we want to use the reference free "helpfulness" evaluator. For it to be "helpful".

In [8]:
from langchain.evaluation import load_evaluator
from langsmith.evaluation import EvaluationResult, RunEvaluator
from langsmith.schemas import Example, Run


class ConversationalHelpfulnessEvaluator(RunEvaluator):
    def __init__(self):
        self.evaluator = load_evaluator("score_string", normalize_by=10)

    def format_conversation(self, question: str, chat_history: list):
        return (
            "\n".join([f"{turn['role']}: {turn['content']}" for turn in chat_history])
            + f"User: {question}"
        )

    def evaluate_run(self, run: Run, example: Example):
        question = run.inputs["question"]
        chat_history = run.inputs["chat_history"]
        prediction = run.outputs["output"]
        formatted_history = self.format_conversation(question, chat_history)
        eval_result = self.evaluator.evaluate_strings(
            input=formatted_history,
            prediction=prediction,
        )
        return EvaluationResult(key="conversation_helpfulness", **eval_result)

## Step 3: Define Chat Bot

We will create a very simple bot for the sake of this tutorial.

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.output_parser import StrOutputParser

# An example chain
chain = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful tutor AI."),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
        ]
    )
    | ChatOpenAI(model="gpt-3.5-turbo")
    | StrOutputParser()
)

In [7]:
from langchain.adapters.openai import convert_openai_messages
from langchain.smith import RunEvalConfig

eval_config = RunEvalConfig(
    custom_evaluators=[ConversationalHelpfulnessEvaluator()],
)


def construct_chain():
    # Add a step to convert the data from the dataset to a form the chain can consume
    return {
        "input": lambda x: x["question"],
        "chat_history": lambda x: convert_openai_messages(x["chat_history"]),
    } | chain


results = client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=construct_chain,
    evaluation=eval_config,
)

View the evaluation results for project 'test-diligent-exchange-9' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/c7de3bd4-1ad7-43d3-8976-8569332741ba?eval=true
[------------------------------------------------->] 5/5